In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import argparse
import sys
import time

from joblib import Parallel, delayed
from utilities2015 import *

%matplotlib inline

In [ ]:
all_cluster_probs = np.exp(-np.asarray(all_cluster_scores)*10.)

In [ ]:
plt.hist(all_cluster_scores, bins=100);
plt.title('distribution of cluster scores');
plt.xlabel('score');
plt.ylabel('number of clusters');
plt.show();

plt.hist(all_cluster_probs, bins=100);
plt.title('distribution of significant cluster probabilities');
plt.xlabel('probability');
plt.ylabel('number of clusters');
plt.show();

In [ ]:
import itertools
from collections import defaultdict

In [ ]:
voters = defaultdict(set)
for (i1, (s1,c1)), (i2, (s2,c2)) in itertools.product(enumerate(zip(all_seed[:1000], all_cluster_dedges[:1000])), 
                                enumerate(zip(all_seed[:1000], all_cluster_dedges[:1000]))):
    common_dedges = set(c1) & set(c2)
    voters[frozenset(common_dedges)].add(frozenset([i1, i2]))

In [ ]:
voters.keys()

In [ ]:
voters_flat = dict([(dedges, set.union(*map(set, pairs))) for dedges, pairs in voters.iteritems()])

In [ ]:
common_dedges_score = [np.asarray(all_cluster_scores)[list(sptr_indices)].mean() for dedges, sptr_indices in voters_flat.iteritems()]

In [ ]:
common_dedges_scores_ranked, common_dedges_ranked = zip(*sorted(zip(common_dedges_score, voters_flat.keys()), reverse=True))

In [ ]:
relevant_dedges = [d for d in common_dedges_ranked if len(d) == 5]

In [ ]:
viz = dm.visualize_edge_sets(common_dedges_ranked[10:20], bg='segmentationWithText', show_set_index=True)
display(viz)

In [ ]:
common_dedges = voters_flat.keys()

In [ ]:
common_dedges_indices_grouped, common_dedges_grouped, _ = group_clusters(common_dedges[:1000], 
                                                                         metric='min-jaccard',
                                                                        dist_thresh=.2)

In [ ]:
common_dedge_group_scores = [np.max([common_dedges_score[i] for i in indices]) for indices in common_dedges_indices_grouped]

In [ ]:
common_dedge_group_representatives = [common_dedges[indices[np.argmax([common_dedges_score[i] for i in indices])]] for indices in common_dedges_indices_grouped]

In [ ]:
common_dedge_group_score_ranked ,\
common_dedge_indices_grouped_ranked,\
common_dedge_group_representatives_ranked, \
common_dedge_groups_ranked = zip(*sorted(zip(common_dedge_group_scores, 
                                             common_dedges_indices_grouped,
                                             common_dedge_group_representatives,
                                             common_dedges_grouped), reverse=True))

In [ ]:
viz = dm.visualize_edge_sets([common_dedges[i] for i in common_dedges_indices_grouped[0]], 
                             bg='segmentationWithText', show_set_index=True)
display(viz)

In [ ]:
viz = dm.visualize_edge_sets(common_dedge_group_representatives_ranked[:100], bg='segmentationWithText', show_set_index=True)
display(viz)

In [ ]:
segment_score_dict = defaultdict(int)
segment_supporter_dict = defaultdict(list)

for seed, cl, prob, dedge_list in zip(all_seed, all_clusters, all_cluster_probs, all_cluster_dedges):
    n_edges = len(dedge_list)
    dummy_list = dedge_list + dedge_list
    for start_edge_ind in range(n_edges):
        segm = dummy_list[start_edge_ind:start_edge_ind+5]
        segment_score_dict[frozenset(segm)] += -np.log(1-.2*prob)
#         segment_score_dict[frozenset(segm)] += sig_score/len(cl)
#         segment_score_dict[frozenset(segm)] += - np.log(1-sig_score)
        
        segment_supporter_dict[frozenset(segm)].append(seed)
        
segment_score_dict.default_factory = None
segment_supporter_dict.default_factory = None

In [ ]:
segment_prob_dict = dict([(segm, 1-np.exp(-score)) for segm, score in segment_score_dict.iteritems()])

# segment_prob_dict = dict([(segm, 1-np.exp(-score/len(sptrs))) for (segm, score), sptrs in zip(segment_score_dict.iteritems(),
#                                                                          segment_supporter_dict.itervalues())])

In [ ]:
plt.hist(segment_score_dict.values(), bins=100);
plt.title('distribution of segment scores');
plt.xlabel('score');
plt.ylabel('number of segments');
plt.show();

plt.hist(segment_prob_dict.values(), bins=100);
# plt.hist(segment_prob_dict.values(), bins=np.linspace(0.9999, 1.0, 100));
plt.title('distribution of segment probs');
plt.xlabel('probability');
plt.ylabel('number of segments');
plt.show();

In [ ]:
plt.hist(map(len, segment_supporter_dict.values()), bins=100);
plt.title('distribution of supporter size of segments');
plt.xlabel('supporter size');
plt.ylabel('number of segments');

In [ ]:
from operator import itemgetter
segment_score_sorted = sorted(segment_score_dict.items(), key=itemgetter(1), reverse=True)
segment_prob_sorted = sorted(segment_prob_dict.items(), key=itemgetter(1), reverse=True)

In [ ]:
good_segments = [set(i[0]) for i in segment_prob_sorted if i[1] > .99]
# good_segments = [set(i[0]) for i in segment_prob_sorted if i[1] > .999]
# good_segments = [set(i[0]) for i in segment_score_sorted if i[1] > 30]
print len(good_segments)

In [ ]:
good_edges = set.union(*good_segments)

In [ ]:
viz = dm.visualize_edge_set(good_edges, bg='segmentationWithText')
display(viz)

In [ ]:
relevant_edges = set.union(*[set(segm) for segm, sptrs in segment_supporter_dict.iteritems() if 2167 in sptrs])

In [ ]:
for segm, sptrs in segment_supporter_dict.iteritems():
    if 2167 in sptrs:
        print segment_score_dict[segm]

In [ ]:
segment_supporters = segment_supporter_dict.values()

In [ ]:
good_segment_supporters = dict((frozenset(seg), segment_supporter_dict[frozenset(seg)]) for seg in good_segments)

In [ ]:
good_segment_groups, _, _ = group_clusters(clusters=good_segment_supporters, metric='jaccard')

In [ ]:
good_edge_groups = [set.union(*map(set, segm_list)) for segm_list in good_segment_groups]

In [ ]:
good_edge_group_scores = [np.mean([segment_score_dict[segm] for segm in segm_list]) for segm_list in good_segment_groups]
good_edge_group_probs = [np.mean([segment_prob_dict[segm] for segm in segm_list]) for segm_list in good_segment_groups]

In [ ]:
plt.hist(good_edge_group_scores, bins=100);
plt.title('distribution of edge group scores');
plt.xlabel('edge group scores');
plt.ylabel('number of groups');
plt.show();

plt.hist(good_edge_group_probs, bins=100);
plt.title('distribution of edge group probs');
plt.xlabel('probability');
plt.ylabel('number of groups');

In [ ]:
good_edge_group_probs_ranked, \
good_edge_group_scores_ranked, \
good_edge_groups_ranked, \
good_edge_group_inices_ranked = zip(*sorted(zip(good_edge_group_probs,
                                                good_edge_group_scores, 
                                                good_edge_groups,
                                                range(len(good_edge_group_scores))), reverse=True))

In [ ]:
viz = dm.visualize_edge_sets(good_edge_groups_ranked[:200], bg='segmentationWithText', show_set_index=True)
display(viz)

In [ ]:
viz = dm.visualize_edge_set({(1649, 1752),
   (1650, 1753),
   (1751, 1752),
   (1754, 1753),
   (1754, 1857),
   (1855, 1752),
   (1855, 1856),
   (1858, 1857),
   (1858, 1961),
   (1958, 1959),
   (1962, 1961),
   (1962, 2065),
   (2061, 2062),
   (2066, 2065),
   (2164, 2165),
   (2169, 2168),
   (2267, 2268),
   (2271, 2168),
   (2271, 2270),
   (2370, 2371),
   (2473, 2474),
   (2475, 2372),
   (2475, 2474),
   (2577, 2474)}, bg='segmentationWithText')
display(viz)

In [ ]:
(1958,1959) in good_edges

In [ ]:
segment_score_dict[frozenset({(1855, 1856),
            (1958, 1959),
            (2061, 2062),
            (2164, 2165),
            (2267, 2268)})]

In [ ]:
[segm for segm in good_segments if (1958, 1959) in segm]

In [ ]:
[segment_score_dict[frozenset(segm)] for segm in good_segments if (1958, 1959) in segm]

In [ ]:
(1958, 1959) in good_edges

In [ ]:
good_edge_group_scores_ranked[462]

In [ ]:
[(rank, segm) for rank, segm in enumerate(good_edge_groups_ranked) if (1958,1959) in segm]

In [ ]:
# votes for directed edgelets
dedge_vote_dict = defaultdict(float)

sys.stderr.write('compute supporter set of each edgelet ...\n')
t = time.time()

# Compute the supporter sets of every edgelet, based on region proposals
# supporter_all[(100,101)] is the set of superpixels that supports directed edgelet (100,101)
dedge_supporters = defaultdict(list)
    
# for s in range(n_superpixels):

for s, c in all_see_cluster_tuples:
#     c = list(expansion_clusters[s])

    interior_texture = texton_hists[c].mean(axis=0)
    b_sps = surrounds_sps[s]
    b_contrasts = cdist(texton_hists[b_sps], interior_texture[np.newaxis, :], chi2)

    for b_sp, b_contrast in zip(b_sps, b_contrasts):
        int_sps = neighbors[b_sp] & set(c)
        for int_sp in int_sps:
            # weight of each edgelet is the contrast normalized by region size
#             weight = float(b_contrast) / max(len(c), 5)
#             weight = 1. / max(len(c), 5)
            weight = 1.
            dedge_vote_dict[(b_sp, int_sp)] += weight
#             dedge_vote_dict[(int_sp, b_sp)] += weight
            dedge_supporters[(b_sp, int_sp)].append(s) # (border_sp, interior_sp) or (out, in)

dedge_vote_dict.default_factory = None
dedge_supporters.default_factory = None
    
sys.stderr.write('done in %f seconds\n' % (time.time() - t))

In [ ]:
all_edges = edge_coords.keys()
all_dedges = set(chain(*[[(i,j),(j,i)] for i,j in all_edges]))

try:
    edge_contained_by = dm.load_pipeline_result('edgeContainedBy', 'pkl')
    print "edgeContainedBy.pkl already exists, skip"

except:

	sys.stderr.write('compute edge-contained-by lookup table...\n')
	t = time.time()

	cluster_edges = dm.load_pipeline_result('clusterEdges', 'pkl')

	def f(c, e):
	    q = set(chain(*[[(i,j),(j,i)] for i,j in combinations(c, 2) if frozenset([i,j]) in all_edges]))
	    return q | set(e)

	contain_edges = Parallel(n_jobs=16)(delayed(f)(c,e) for c, e in zip(expansion_clusters, cluster_edges))

	edge_contained_by = defaultdict(set)
	for sp, es in enumerate(contain_edges):
	    for e in es:
	        edge_contained_by[e].add(sp)

	edge_contained_by.default_factory = None

# 	dm.save_pipeline_result(edge_contained_by, 'edgeContainedBy', 'pkl')

	sys.stderr.write('done in %f seconds\n' % (time.time() - t))

In [ ]:
# only consider dedges that receive non-zero vote
nz_dedges = dedge_vote_dict.keys()

sys.stderr.write('filter dedges ...\n')
t = time.time()

# compute contrast of each dedge
# dedge_contrast = dict([((i,j), chi2(texton_hists[i], texton_hists[j])) for i,j in all_dedges])
dedge_contrast = dict([((i,j), chi2(texton_hists[i], texton_hists[dedge_supporters[(i,j)]].mean(axis=0))) 
                       for i,j in nz_dedges])

# filter dedges, require contrast > .5 and contained by at least 4 growed regions
nz_dedges2 = [e for e, sps in edge_contained_by.iteritems() if len(sps) > 3 and e in nz_dedges]
nz_dedges2 = [e for e in nz_dedges2 if dedge_contrast[e] > .5]

# compute stop ratio of each dedge
dedge_stopperness = dict([(e, dedge_vote_dict[e]/len(edge_contained_by[e])) for e in nz_dedges2])

# filter dedges, require stop ratio to be 1.
nz_dedges2 = [e for e in nz_dedges2 if dedge_stopperness[e] > .8]
#     print len(nz_dedges2), 'valid edges'

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

In [ ]:
len(nz_dedges2)

In [ ]:
t = time.time()
viz = dm.visualize_edge_sets([nz_dedges2], directed=False, colors=np.array([[255,0,0]]),
                            bg='segmentationWithText')
print time.time() - t

In [ ]:
display(viz)

In [ ]:

sys.stderr.write('compute expanded supporter set for each edgelet ...\n')

# find union supporter set for each dedge
dedge_expandedSupporters = dict([(e, smart_union([expansion_clusters[s] for s in dedge_supporters[e]])) 
                             for e in nz_dedges2])

# cluster union supporter sets
dedges_grouped, dedge_supporters_grouped, _ = group_clusters(clusters=dict((e, dedge_expandedSupporters[e]) for e in nz_dedges2),
                                                             dist_thresh=.01, linkage='complete', metric='jaccard')
dedges_grouped = map(set, dedges_grouped)
ngroup = len(dedges_grouped)

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

#     print len(dedges_grouped), 'edge groups'

sys.stderr.write('compute supporter set consistency factor...\n')
t = time.time()
# compute cluster "centroids"
dedge_group_supporters = map(smart_union, dedge_supporters_grouped)

# compute centroid distances
# analyze supporter set consistency factor (1)
dedge_group_supporter_distmat = compute_pairwise_distances(dedge_group_supporters, metric='jaccard')
np.fill_diagonal(dedge_group_supporter_distmat, 0)

sys.stderr.write('done in %f seconds\n' % (time.time() - t))


sys.stderr.write('compute connectivity factor...\n')
t = time.time()

# analyze connectivity factor (2)
G = networkx.from_dict_of_lists(dedge_neighbors)
conns = [[set() if any([sorted(e1)==sorted(e2) for e1, e2 in product(eg1, eg2)]) 
          else set([(i,j) for i,j in G.edges(eg1|eg2) if (i in eg1 and j in eg2) or (j in eg1 and i in eg2)]) 
         for eg1 in dedges_grouped] for eg2 in dedges_grouped]
conns_flat = [a for b in conns for a in b ]
dedge_group_edgeConn_distmat = np.reshape(map(lambda x: len(x) < 1, conns_flat), (ngroup, ngroup))
np.fill_diagonal(dedge_group_edgeConn_distmat, 0)

sys.stderr.write('done in %f seconds\n' % (time.time() - t))


sys.stderr.write('compute connectivity factor...\n')
t = time.time()

# analyze texture similarity factor (3)
dedge_group_supporterTex_distmat = np.reshape([chi2(texton_hists[list(sps1)].mean(axis=0), texton_hists[list(sps2)].mean(axis=0))  
		for sps1, sps2 in product(dedge_group_supporters, dedge_group_supporters)], (ngroup, ngroup))

sys.stderr.write('done in %f seconds\n' % (time.time() - t))


sys.stderr.write('combine three factors ...\n')
t = time.time()

# combine above three factors
dedge_group_distmat = 1 - (1-dedge_group_edgeConn_distmat) * (1-dedge_group_supporter_distmat>0.1) * (dedge_group_supporterTex_distmat < .25)

sys.stderr.write('done in %f seconds\n' % (time.time() - t))


sys.stderr.write('further cluster dedge groups ...\n')
t = time.time()

# further cluster dedge groups
_, edge_groups, _ = group_clusters(clusters=dedges_grouped, 
                               distance_matrix=dedge_group_distmat, 
                               dist_thresh=.5, linkage='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))

#     print len(edge_groups), 'edge groups after considering connectivity'

edge_groups = map(lambda x: set(chain(*x)), edge_groups)

# sort clusters by total contrast
edge_groups_sorted = sorted(edge_groups, key=lambda x: sum(dedge_contrast[e] for e in x), reverse=True)
edge_group_supporters_sorted = [smart_union(map(lambda e: dedge_expandedSupporters[e], es)) 
                                for es in edge_groups_sorted]


viz = dm.visualize_edge_sets(edge_groups_sorted[:40], text_size=3, img=segmentation_vis)
dm.save_pipeline_result(viz, 'topLandmarks', 'jpg')

dm.save_pipeline_result(edge_groups_sorted, 'goodEdgeSets', 'pkl')
dm.save_pipeline_result(edge_group_supporters_sorted, 'goodEdgeSetsSupporters', 'pkl')



boundary_models = []

for i, es in enumerate(good_dedges):

    es = list(es)

    interior_texture = texton_hists[list(good_clusters[i])].mean(axis=0)

    surrounds = [e[0] for e in es]
    exterior_textures = np.array([texton_hists[s] if s!=-1 else np.nan * np.ones((texton_hists.shape[1],)) 
                                  for s in surrounds])
    # how to deal with -1 in surrounds? Assign to an all np.nan vector

    points = np.array([edge_coords[frozenset(e)].mean(axis=0) for e in es])
    center = points.mean(axis=0)

    boundary_models.append((es, interior_texture, exterior_textures, points, center))    